In [1]:
import os

from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Load environment variables from .env
load_dotenv()

True

In [2]:
# Define the persistent directory
current_dir = os.getcwd()
persistent_directory = os.path.join(
    current_dir, "db", "chroma_db_with_metadata")

In [3]:
# Define the embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

/tmp/ipykernel_7855/1099210415.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=persistent_directory,


In [4]:
# Define the user's question
query = "How can I learn more about LangChain?"

In [5]:
# Retrieve relevant documents based on the query
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)
relevant_docs = retriever.invoke(query)

In [6]:
# Display the relevant results with metadata
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")


--- Relevant Documents ---
Document 1:
“The leader of the gang. I shall have the others, but he first.”

“How did you trace it, then?”

He took a large sheet of paper from his pocket, all covered with dates
and names.

“I have spent the whole day,” said he, “over Lloyd’s registers and
files of the old papers, following the future career of every vessel
which touched at Pondicherry in January and February in ’83. There were
thirty-six ships of fair tonnage which were reported there during those
months. Of these, one, the _Lone Star_, instantly attracted my
attention, since, although it was reported as having cleared from
London, the name is that which is given to one of the states of the
Union.”

“Texas, I think.”

“I was not and am not sure which; but I knew that the ship must have an
American origin.”

“What then?”

Document 2:
Section 3. Information about the Project Gutenberg Literary Archive Foundation

The Project Gutenberg Literary Archive Foundation is a non-profit
501(c)(3) ed

In [7]:
# Combine the query and the relevant document contents
combined_input = (
    "Here are some documents that might help answer the question: "
    + query
    + "\n\nRelevant Documents:\n"
    + "\n\n".join([doc.page_content for doc in relevant_docs])
    + "\n\nPlease provide an answer based only on the provided documents. If the answer is not found in the documents, respond with 'I'm not sure'."
)


In [12]:
# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o-mini")

# Define the messages for the model
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=combined_input),
]


In [13]:
# Invoke the model with the combined input
result = model.invoke(messages)

In [14]:
# Display the full result and content only
print("\n--- Generated Response ---")
# print("Full result:")
# print(result)
print("Content only:")
print(result.content)


--- Generated Response ---
Content only:
I'm not sure.
